In [286]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [288]:
df=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\anu course\datasets\bmarketing.csv")

In [289]:
df['y']=df['y'].map({'yes':1,'no':0})

In [290]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object

In [291]:
x=df.drop('y',axis=1)
y=df['y']

In [292]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)

In [293]:
numerical_features = x.select_dtypes(include=['int64','float64']).columns.tolist()
print(numerical_features)

['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


In [294]:
categorical_features = x.select_dtypes(include=['object']).columns.tolist()
print(categorical_features)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [295]:
numerical_cols = Pipeline(
    steps=[("Simple Imputer",SimpleImputer(strategy='mean')),
           ("Scaling",StandardScaler())]
)

In [296]:
categorical_cols = Pipeline(
    steps=[("Simple Imputer",SimpleImputer(strategy='most_frequent')),
           ("ohe",OneHotEncoder(handle_unknown='ignore'))]
)

In [297]:
preprocessing = ColumnTransformer(
    transformers= [("numerical",numerical_cols,numerical_features),
                   ("categorical",categorical_cols,categorical_features)]
)

In [298]:
lor_pipeline = Pipeline(
    steps = [("preprocessing",preprocessing),
             ("model",LogisticRegression())]
)

In [299]:
lor_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['age', 'duration',
                                                   'campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('model', LogisticRegression())])

In [300]:
svr_pipeline = Pipeline(
    steps = [("preprocessing",preprocessing),
            ("model",SVC())]
)

In [301]:
svr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['age', 'duration',
                                                   'campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('model', SVC())])

In [302]:
mlp_pipeline = Pipeline(
    steps = [("preprocessing",preprocessing),
            ("model",MLPClassifier())]
)

In [303]:
mlp_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['age', 'duration',
                                                   'campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('model', MLPClassifier())])

In [304]:
knn_pipeline = Pipeline(
    steps = [("preprocessing",preprocessing),
            ("model",KNeighborsClassifier())]
)

In [305]:
knn_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer()),
                                                                  ('Scaling',
                                                                   StandardScaler())]),
                                                  ['age', 'duration',
                                                   'campaign', 'pdays',
                                                   'previous', 'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('categorical',
                                                  Pipeline(steps=[('Simple '
                                                                   'Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('model', KNeighborsClassifier())])

In [306]:
models = {
    "logistic regression":lor_pipeline,
    "Support Vector Machine":svr_pipeline,
    "MultiLayer Perceptron":mlp_pipeline,
    "K Nearest Neighbors":knn_pipeline
}

In [307]:
results={}

In [308]:
for name, model in models.items():   
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name}: {acc*100:.2f}%")

logistic regression: 91.05%
Support Vector Machine: 91.15%


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MultiLayer Perceptron: 90.31%
K Nearest Neighbors: 90.02%
